In [1]:
import pandas as pd
import numpy as np
from scipy.io import loadmat
from sklearn.metrics import pairwise_distances

In [2]:
# path for mats and label.csv
path = '/train_set/'

In [3]:
train_pt_dir = path + "points/"
train_label_path = path + "label.csv"

In [4]:
label_df = pd.read_csv(train_label_path)
label_df.head()

,Unnamed: 0,Index,identity,emotion_idx,emotion_cat,type
0,1,1,111,1,Neutral,simple
1,2,2,114,1,Neutral,simple
2,3,3,115,1,Neutral,simple
3,4,4,116,1,Neutral,simple
4,5,5,117,1,Neutral,simple


In [5]:
# import matlab matrix and round to int
index = ['{0:04}'.format(num) for num in range(1, 2501)]

mats = []
for ind in index:
    temp = loadmat( train_pt_dir + ind + ".mat")
    mats.append(temp[[*temp][-1]])
    
mats = [mat.round() for mat in mats]

In [6]:
# original feature function 78*77
def feature_distance(mat_list, nfidu = 78):
    def pairwise_dist(vec):
        a = pairwise_distances(vec.reshape(nfidu,1))
        return(a[np.triu_indices(nfidu, k = 1)])
    
    def pairwise_dist_result(mat_list):
        a = np.apply_along_axis(pairwise_dist, 0, mat_list)
        return(a.flatten('F')) 
     
    feature_mat = [pairwise_dist_result(mat) for mat in mat_list]   
    return(np.vstack(feature_mat))

In [7]:
feature_dist = feature_distance(mats)

In [8]:
feature_dist_df = pd.DataFrame(feature_dist)
feature_dist_df.columns = ["feature"+ str(num) for num in range(1, 6007)]

In [9]:
feature_dist_df.head()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature5997,feature5998,feature5999,feature6000,feature6001,feature6002,feature6003,feature6004,feature6005,feature6006
0,45.0,28.0,6.0,16.0,37.0,22.0,1.0,25.0,176.0,136.0,...,55.0,111.0,168.0,225.0,56.0,113.0,170.0,57.0,114.0,57.0
1,44.0,27.0,5.0,19.0,36.0,20.0,2.0,24.0,170.0,134.0,...,53.0,111.0,169.0,228.0,58.0,116.0,175.0,58.0,117.0,59.0
2,42.0,24.0,3.0,18.0,33.0,15.0,5.0,23.0,153.0,118.0,...,49.0,102.0,156.0,211.0,53.0,107.0,162.0,54.0,109.0,55.0
3,47.0,27.0,7.0,15.0,31.0,13.0,8.0,29.0,166.0,126.0,...,52.0,112.0,173.0,234.0,60.0,121.0,182.0,61.0,122.0,61.0
4,35.0,22.0,1.0,19.0,36.0,19.0,0.0,18.0,161.0,122.0,...,50.0,102.0,157.0,211.0,52.0,107.0,161.0,55.0,109.0,54.0


In [10]:
# new feature: the degree angle between each vector and x-axis, range:0-90
# 78*77/2
def feature_slope(mat_list, nfidu = 78):
    def pairwise_dist(vec):
        a = pairwise_distances(vec.reshape(nfidu,1))
        return(a[np.triu_indices(nfidu, k = 1)])
    
    def pairwise_dist_result(mat):
        a = np.apply_along_axis(pairwise_dist, 0, mat)
        return(np.rad2deg(np.arctan2(a[:,1], a[:,0]))) 
     
    feature_mat = [pairwise_dist_result(mat) for mat in mat_list]   
    return(np.vstack(feature_mat))

In [11]:
feature_slope_df = pd.DataFrame(feature_slope(mats))
feature_slope_df.columns = ["feature"+ str(num) for num in range(1, 3004)]

In [12]:
feature_slope_df.head()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature2994,feature2995,feature2996,feature2997,feature2998,feature2999,feature3000,feature3001,feature3002,feature3003
0,5.079608,19.653824,65.224859,36.869898,9.211027,28.610460,86.185925,29.248826,0.000000,2.526117,...,70.016893,72.030860,74.687182,80.166436,74.054604,77.042420,83.622819,80.049373,88.492564,82.998732
1,15.255119,8.426969,65.556045,25.346176,9.462322,46.397181,85.236358,41.185925,1.684684,5.964487,...,66.540976,72.969843,77.005383,79.805763,79.261103,82.146687,84.127372,85.072890,86.576129,88.058514
2,6.788975,14.036243,73.300756,18.434949,13.626995,48.576334,75.256437,29.475889,0.374477,5.325741,...,63.904576,71.565051,77.005383,80.846191,79.315088,83.601043,86.467705,87.878904,90.000000,87.917435
3,2.436648,16.504361,63.434949,36.253838,5.527540,40.236358,53.972627,20.772255,0.345151,3.632951,...,63.434949,71.718659,76.354049,79.587455,79.611142,82.467399,84.663680,85.314100,87.184443,89.060809
4,15.945396,10.304846,80.537678,11.888658,11.003541,38.290163,90.000000,45.000000,0.711712,5.152082,...,66.250506,70.559965,74.344218,77.183998,74.931512,78.381476,80.826465,81.724107,83.717508,85.763605
